In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [77]:
import pandas as pd

wordList = {}
spam = pd.read_csv('./gdrive/MyDrive/spam-words.txt', encoding='utf-8')

def word_list(text):
  indx = len(wordList)

  for word in text:
    wordList[word] = indx
    indx += 1

if __name__ == '__main__':
  for i in range(spam.shape[0]):
    text = spam.iloc[i,0].split()
  print('the length of vokabulary is ', len(wordList))

  word_list(text)

file = open("word_list.txt", "w")
file.write(str(wordList))
file.close()

the length of vokabulary is  0


In [90]:
import numpy as np
import pandas as pd
import ast

data = pd.read_csv('./gdrive/MyDrive/sms_data.txt', encoding='utf-16')
file = open('word_list.txt', 'r', encoding='utf-8')
cntnt = file.read()
word_list = ast.literal_eval(cntnt)

x = np.zeros((data.shape[0], len(word_list)))
y = np.zeros((data.shape[0]))

for i in range(data.shape[0]):
  sms = data.iloc[i,0].split()

for sms_word in sms:
  if sms_word.lower() in word_list:
    x[i, word_list[sms_word]] +=1
    #y[i] = data.iloc[i, 0]

np.save('/content/gdrive/MyDrive/dataset/x.npy', x)
np.save('/content/gdrive/MyDrive/dataset/y.npy', y)

In [92]:
import numpy as np

class naivebayes():
  def __init__(self, X, y):
    self.num_examples, self.num_features = X.shape
    self.num_classes = len(np.unique(y))
    self.eps = 1e-6
  def fit(self, X, y):
    self.classes_mean = {}
    self.classes_variance = {}
    self.classes_prior = {}

    for c in range(self.num_classes):
      X_c = X[y==c]
      self.classes_mean[str(c)] = np.mean(X_c, axis=0)
      self.classes_variance[str(c)] = np.var(X_c, axis=0)
      self.classes_prior[str(c)] = X_c.shape[0]/self.num_examples

  def predict(self, X):
    probs = np.zeros((self.num_examples, self.num_classes))

    for c in range(self.num_classes):
      prior = self.classes_prior[str(c)]
      probs_c = self.density_function(X, self.classes_mean[str(c)], self.classes_variance[str(c)])
      probs[:, c] = probs_c +np.log(prior)

    return np.argmax(probs, 1)

  def density_function(self, X, mean, sigma):
    const = -self.num_features/2 * np.log(2*np.pi) - 0.5*np.sum(sigma+self.eps)
    probs = 0.5*np.sum(np.power(x-mean, 2)/(sigma+self.eps), 1)
    return const - probs
 
  if __name__ == '__main__':
    X = np.load('/content/gdrive/MyDrive/dataset/x.npy')
    y = np.load('/content/gdrive/MyDrive/dataset/y.npy')
    print(X.shape)
    print(y.shape)

    NB = naivebayes(X, y)
    NB.fit(X, y)
    y_pred = NB.predict(X)

print(sum(y_pred == y)/X.shape[0])

(603, 1)
(603,)
1.0
